In [1]:
import os
import torch
import numpy as np
import pandas as pd
import sys
import random
import h5py
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

sys.path.append('../')

from models.smc.src import *
from models.smc.src.crowd_count import *
from models.smc.src.network import *
from models.smc.src.data_loader import ImageDataLoader
from models.smc.src.timer import *
from models.smc.src.evaluate_model import *
from models.smc.src import utils

torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
# Check to see if device can be trained on GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(device)


cuda


In [3]:
# Cuda configurations

torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

current_device = torch.cuda.current_device()
current_device_name = torch.cuda.get_device_name(current_device)
memory_allocated = torch.cuda.memory_allocated()
memory_cached = torch.cuda.memory_reserved()

print(
    f'Using gpu {current_device_name} with device number {current_device}.\n'
    f'Memory allocated = {memory_allocated}\n'
    f'Memory cached = {memory_cached}'
)

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [4]:
try:
    from termcolor import cprint
except ImportError:
    cprint = None


def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)

In [5]:
# Directory Configurations

method = 'smc'
dataset_name = 'JHU'
output_dir = f'../output/{method}/saved_models/{dataset_name}'

# Training data path
train_path = '../data/JHU/train/consolidated'
train_gt_path = '../data/JHU/train/gt'

# Validation data path
val_path = '../data/JHU/val/consolidated'
val_gt_path = '../data/JHU/val/gt'

# Test data path
test_path = '../data/JHU/test/consolidated'
test_gt_path = '../data/JHU/test/gt'

In [6]:
# Create output directory if it doesnt exist

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [7]:
# load model

is_cuda = True  # Determine if we should use the CPU to train or GPU

model = CrowdCounter(is_cuda=is_cuda)  # is_cuda determines if all the input tensors should be converted to cuda tensors
network.weights_normal_init(model, dev=0.01)
model.train()


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


CrowdCounter(
  (model): SMC(
    (r1): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
        (relu): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): Conv2d(
        (conv): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(
        (conv): Conv2d(32, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (5): Conv2d(
        (conv): Conv2d(16, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
    )
    (r2_1): Sequential(
      (0): Conv2d(
        (conv): Conv2d(3, 20, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
        (relu): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_s

In [8]:
# Model parameters

for name, param in model.named_parameters():
    print(f'{name}\t{param.device}\t{param.shape}')

# Print model's state_dict
print("\nModel's state_dict: \n")
for k, v in model.state_dict().items():
    print(k, "\t", v.dtype)

model.r1.0.conv.weight	cpu	torch.Size([16, 3, 9, 9])
model.r1.0.conv.bias	cpu	torch.Size([16])
model.r1.2.conv.weight	cpu	torch.Size([32, 16, 7, 7])
model.r1.2.conv.bias	cpu	torch.Size([32])
model.r1.4.conv.weight	cpu	torch.Size([16, 32, 7, 7])
model.r1.4.conv.bias	cpu	torch.Size([16])
model.r1.5.conv.weight	cpu	torch.Size([8, 16, 7, 7])
model.r1.5.conv.bias	cpu	torch.Size([8])
model.r2_1.0.conv.weight	cpu	torch.Size([20, 3, 7, 7])
model.r2_1.0.conv.bias	cpu	torch.Size([20])
model.r2_1.2.conv.weight	cpu	torch.Size([40, 20, 5, 5])
model.r2_1.2.conv.bias	cpu	torch.Size([40])
model.r2_1.4.conv.weight	cpu	torch.Size([20, 40, 5, 5])
model.r2_1.4.conv.bias	cpu	torch.Size([20])
model.r2_1.5.conv.weight	cpu	torch.Size([10, 20, 5, 5])
model.r2_1.5.conv.bias	cpu	torch.Size([10])
model.r2_2.0.conv.weight	cpu	torch.Size([24, 3, 5, 5])
model.r2_2.0.conv.bias	cpu	torch.Size([24])
model.r2_2.2.conv.weight	cpu	torch.Size([48, 24, 3, 3])
model.r2_2.2.conv.bias	cpu	torch.Size([48])
model.r2_2.4.conv.wei

In [9]:
# Change model weights tensors to be cuda tensors if is_cuda is true and cuda is available

if is_cuda and torch.cuda.is_available():
    print("Changing to cuda weights")
    model.cuda()

Changing to cuda weights


In [10]:
#training configuration

disp_interval = 10

train_loss = 0
re_cnt = False
t = Timer()
t.tic()

# Set initial values
best_mae, best_mse, best_epoch, best_mape = 999999, 999999, 0, 9999999


In [11]:
# Hyperparameters

learning_rate = 0.0001
epochs = 100

# construct an optimizer

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=learning_rate)


In [12]:
# Load the images, take note the num_pool argument

data_loader = ImageDataLoader(train_path, shuffle=False, pre_load=False, size = 200)
data_loader_val = ImageDataLoader(val_path, shuffle=False, pre_load=False, size = 40)
data_loader_test = ImageDataLoader(test_path, shuffle=False, pre_load=False, size= 40)

print('Training instances: {}'.format(data_loader.get_num_samples()))
print('Validation instances: {}'.format(data_loader_val.get_num_samples()))
print('Test instances: {}'.format(data_loader_test.get_num_samples()))

# Saved weights
save_name = None


Training instances: 200
Validation instances: 40
Test instances: 40


In [13]:
from torch.autograd import Variable

current_date = datetime.now()
 
# dd/mm/YY H:M:S
cd_string = current_date.strftime("%d/%m/%Y %H:%M:%S")
#Tensorboard  config
use_tensorboard = False

writer = SummaryWriter(f'../output/tensorboard/runs/{learning_rate}_{epochs}_{cd_string}')

# Add model to tensorboard
dummy_input = data_loader.get_test_input()

#writer.add_graph(model.get_model(), Variable(dummy_input['data'].type(torch.FloatTensor)).cuda())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [14]:
for epoch in range(epochs):    
    step = -1
    train_loss = 0

    for _ , blob in enumerate(data_loader, 1):         
        step = step + 1        
        im_data = blob['data']
        gt_data = blob['gt_density']
                
        # Forward pass + backward pass + optimise
        try:
            density_map = model(im_data, gt_data)
            loss = model.loss
            # Set the parameter gradients to zero
            optimizer.zero_grad()
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()

            # Write to tensorboard
            train_loss += loss.item()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)

            # Write to tensorboard
            #writer.add_scalar(f"Loss/train_{epoch}", train_loss, step)

            if step % disp_interval == 0:
                duration = t.toc(average=False)
                gt_count = torch.sum(gt_data)    

                et_count = torch.sum(density_map)
                #utils.save_results(im_data,gt_data,density_map, output_dir)

                # Log the results per display rate
                log_text = (
                    'epoch: %4d, step %4d, Time: %.4fs, gt_cnt: %5.1f, et_cnt: %5.1f train_loss: %4.1f' % (epoch,
                    step, duration, gt_count,et_count, train_loss)
                )
                log_print(log_text, color='green', attrs=['bold'])

                # Reset timer
                re_cnt = True    


            if re_cnt:                                
                t.tic()
                re_cnt = False

        except Exception as e:
            #print(e)
            continue
            
    continue
    
    # Overwrite the current model weights
    current_model = f'{method}_{learning_rate}.h5'
    save_name = os.path.join(output_dir, current_model)
    network.save_net(save_name, model)

    # Evaluate the mae and mse results by doing a forward pass against the validation dataset i.e data_loader_val for
    # each epoch

    MAEcrowddensity, MSEcrowddensity, MAPEcrowddensity, MAEweather, MSEweather, MAPEweather, MAE, MSE, RMSE, MAPE = evaluate_model(save_name, data_loader_val, is_cuda=False)    
    
    # Pocket algorithm: Check to see if the current epoch mae is better than the best recorded one,
    # If it is, then overwrite the current best .h5 weights file
    if MAE < best_mae:
        # Save the new best mae and mse
        best_mae = MAE
        best_epoch = epoch
        best_model = f'best_MAE_epoch_{epoch}_{method}_{learning_rate}.h5'

        # Overwrite or create a new file for the best model for this learning rate
        save_name = os.path.join(output_dir, best_model)
        best_mae_path = save_name
        network.save_net(save_name, model)
        
    if MSE < best_mse:
        # Save the new best mae and mse
        best_mse = MSE
        best_epoch = epoch

        best_model = f'best_MSE_epoch_{epoch}_{method}_{learning_rate}.h5'

        # Overwrite or create a new file for the best model for this learning rate
        save_name = os.path.join(output_dir, best_model)
        best_mse_path = save_name
        network.save_net(save_name, model)
        
    if MAPE < best_mape:
        # Save the new best mae and mse
        best_mape = MAPE
        best_epoch = epoch
        best_model = f'best_MAPE_epoch_{epoch}_{method}_{learning_rate}.h5'

        # Overwrite or create a new file for the best model for this learning rate
        save_name = os.path.join(output_dir, best_model)
        best_mape_path = save_name
        network.save_net(save_name, model)
        


    # Print out the best epoch that beat the current best mae
    log_text = f'EPOCH: {epoch}, MAE: {MAE}, MSE: {MSE} RMSE: {RMSE}'
    log_print(log_text, color='green', attrs=['bold'])


    # Save the results to tensorboard for each epoch
    if use_tensorboard:
        # overall segment
        writer.add_scalar("Overall/Val MAE", MAE, epoch)
        writer.add_scalar("Overall/Val MSE", MSE, epoch)
        writer.add_scalar("Overall/Val RMSE", RMSE, epoch)
        writer.add_scalar("Overall/Train MSE", train_loss / data_loader.get_num_samples(), epoch)
        writer.add_scalar("Overall/Train RMSE", np.sqrt(train_loss / data_loader.get_num_samples()), epoch)
        writer.add_scalar("Overall/Val MAPE", MAPE, epoch)
        writer.add_scalar("Overall/Train Loss", train_loss / data_loader.get_num_samples(), epoch)
        
        
        # crowd density segment

        writer.add_scalar('Crowd Density/High/MAE', MAEcrowddensity['High'], epoch)
        writer.add_scalar('Crowd Density/High/MSE', MSEcrowddensity['High'], epoch)
        writer.add_scalar('Crowd Density/High/RMSE', np.sqrt(MSEcrowddensity['High']), epoch)
        writer.add_scalar('Crowd Density/High/MAPE', MAPEcrowddensity['High'], epoch)
        
        writer.add_scalar('Crowd Density/Med/MAE', MAEcrowddensity['Med'], epoch)
        writer.add_scalar('Crowd Density/Med/MSE', MSEcrowddensity['Med'], epoch)
        writer.add_scalar('Crowd Density/Med/RMSE', np.sqrt(MSEcrowddensity['Med']), epoch)
        writer.add_scalar('Crowd Density/Med/MAPE', MAPEcrowddensity['Med'], epoch)
        
        writer.add_scalar('Crowd Density/Low/MAE', MAEcrowddensity['Low'], epoch)
        writer.add_scalar('Crowd Density/Low/MSE', MSEcrowddensity['Low'], epoch)
        writer.add_scalar('Crowd Density/Low/RMSE', np.sqrt(MSEcrowddensity['Low']), epoch)
        writer.add_scalar('Crowd Density/Low/MAPE', MAPEcrowddensity['Low'], epoch)
        
        # weather segment

        writer.add_scalar('Weather/No Degradation/MAE', MAEweather['None'], epoch)
        writer.add_scalar('Weather/No Degradation/MSE', MSEweather['None'], epoch)
        writer.add_scalar('Weather/No Degradation/RMSE', np.sqrt(MSEweather['None']), epoch)
        writer.add_scalar('Weather/No Degradation/MAPE', MAPEweather['None'], epoch)
        
        writer.add_scalar('Weather/Fog/MAE', MAEweather['Fog'], epoch)
        writer.add_scalar('Weather/Fog/MSE', MSEweather['Fog'], epoch)
        writer.add_scalar('Weather/Fog/RMSE', np.sqrt(MSEweather['Fog']), epoch)
        writer.add_scalar('Weather/Fog/MAPE', MAPEweather['Fog'], epoch)
        
        writer.add_scalar('Weather/Rain/MAE', MAEweather['Rain'], epoch)
        writer.add_scalar('Weather/Rain/MSE', MSEweather['Rain'], epoch)
        writer.add_scalar('Weather/Rain/RMSE', np.sqrt(MSEweather['Rain']), epoch)
        writer.add_scalar('Weather/Rain/MAPE', MAPEweather['Rain'], epoch)
        

        writer.add_scalar('Weather/Snow/MAE', MAEweather['Snow'], epoch)
        writer.add_scalar('Weather/Snow/MSE', MSEweather['Snow'], epoch)
        writer.add_scalar('Weather/Snow/RMSE', np.sqrt(MSEweather['Snow']), epoch)
        writer.add_scalar('Weather/Snow/MAPE', MAPEweather['Snow'], epoch)

epoch:    0, step    0, Time: 2.1806s, gt_cnt:  40.0, et_cnt:   0.1 train_loss: 43.3
epoch:    0, step   10, Time: 28.1840s, gt_cnt: 120.0, et_cnt:  25.0 train_loss: 1626.1
epoch:    0, step   20, Time: 21.2665s, gt_cnt:  47.0, et_cnt: 175.8 train_loss: 2174.9
epoch:    0, step   40, Time: 32.8532s, gt_cnt:  38.0, et_cnt: 199.9 train_loss: 9483.0
epoch:    0, step   50, Time: 16.2973s, gt_cnt:  62.0, et_cnt:  32.3 train_loss: 9961.0
epoch:    0, step   60, Time: 22.5281s, gt_cnt: 560.0, et_cnt: 153.0 train_loss: 13320.7
epoch:    0, step   70, Time: 13.7088s, gt_cnt:  76.0, et_cnt:  23.3 train_loss: 15070.2
epoch:    0, step   90, Time: 27.7778s, gt_cnt:  85.0, et_cnt: 124.7 train_loss: 17578.8
epoch:    0, step  100, Time: 15.5982s, gt_cnt:  61.0, et_cnt:  87.4 train_loss: 18173.1
epoch:    0, step  110, Time: 15.4500s, gt_cnt:  31.0, et_cnt: 351.0 train_loss: 20877.4
epoch:    0, step  120, Time: 10.0393s, gt_cnt: 196.0, et_cnt:  90.3 train_loss: 22668.7
epoch:    0, step  130, Time:

epoch:    4, step  110, Time: 14.9593s, gt_cnt:  31.0, et_cnt: 451.2 train_loss: 20843.5
epoch:    4, step  120, Time: 9.5388s, gt_cnt: 196.0, et_cnt: 104.0 train_loss: 22628.6
epoch:    4, step  130, Time: 19.3561s, gt_cnt:  42.0, et_cnt:  49.7 train_loss: 23339.9
epoch:    4, step  140, Time: 25.8738s, gt_cnt: 678.0, et_cnt:  60.4 train_loss: 30726.0
epoch:    4, step  150, Time: 19.7234s, gt_cnt: 1622.0, et_cnt: 566.2 train_loss: 33889.5
epoch:    4, step  160, Time: 20.3668s, gt_cnt:  20.0, et_cnt: 101.1 train_loss: 35952.5
epoch:    4, step  170, Time: 15.5530s, gt_cnt: 227.0, et_cnt: 140.5 train_loss: 37518.0
epoch:    4, step  180, Time: 13.0742s, gt_cnt:  95.0, et_cnt: 107.0 train_loss: 40123.0
epoch:    4, step  190, Time: 10.1409s, gt_cnt: 273.0, et_cnt: 238.3 train_loss: 42021.0
epoch:    5, step    0, Time: 11.3796s, gt_cnt:  40.0, et_cnt: 182.7 train_loss: 43.9
epoch:    5, step   10, Time: 30.3483s, gt_cnt: 120.0, et_cnt:  63.2 train_loss: 1627.9
epoch:    5, step   20, T

epoch:    9, step    0, Time: 11.4119s, gt_cnt:  40.0, et_cnt: 215.8 train_loss: 44.7
epoch:    9, step   10, Time: 30.6202s, gt_cnt: 120.0, et_cnt:  52.6 train_loss: 1632.7
epoch:    9, step   20, Time: 21.7119s, gt_cnt:  47.0, et_cnt: 116.3 train_loss: 2180.7
epoch:    9, step   40, Time: 35.9369s, gt_cnt:  38.0, et_cnt: 247.1 train_loss: 9463.3
epoch:    9, step   50, Time: 19.4082s, gt_cnt:  62.0, et_cnt:  56.5 train_loss: 9938.7
epoch:    9, step   60, Time: 22.1416s, gt_cnt: 560.0, et_cnt: 366.2 train_loss: 13272.0
epoch:    9, step   70, Time: 13.6113s, gt_cnt:  76.0, et_cnt:  54.8 train_loss: 15018.1
epoch:    9, step   90, Time: 32.4897s, gt_cnt:  85.0, et_cnt: 101.2 train_loss: 17515.2
epoch:    9, step  100, Time: 16.4997s, gt_cnt:  61.0, et_cnt:  60.4 train_loss: 18107.9
epoch:    9, step  110, Time: 14.9391s, gt_cnt:  31.0, et_cnt: 235.4 train_loss: 20788.2
epoch:    9, step  120, Time: 9.5995s, gt_cnt: 196.0, et_cnt: 111.3 train_loss: 22567.1
epoch:    9, step  130, Time:

epoch:   13, step  110, Time: 14.8932s, gt_cnt:  31.0, et_cnt: 182.3 train_loss: 20749.8
epoch:   13, step  120, Time: 9.5983s, gt_cnt: 196.0, et_cnt: 107.9 train_loss: 22524.6
epoch:   13, step  130, Time: 19.8146s, gt_cnt:  42.0, et_cnt:  68.2 train_loss: 23249.2
epoch:   13, step  140, Time: 26.2805s, gt_cnt: 678.0, et_cnt:  92.0 train_loss: 30592.6
epoch:   13, step  150, Time: 19.9786s, gt_cnt: 1622.0, et_cnt: 1047.2 train_loss: 33727.6
epoch:   13, step  160, Time: 20.2382s, gt_cnt:  20.0, et_cnt:  72.3 train_loss: 35780.4
epoch:   13, step  170, Time: 15.7386s, gt_cnt: 227.0, et_cnt: 186.9 train_loss: 37347.9
epoch:   13, step  180, Time: 13.2941s, gt_cnt:  95.0, et_cnt: 148.2 train_loss: 39924.1
epoch:   13, step  190, Time: 10.2021s, gt_cnt: 273.0, et_cnt: 179.7 train_loss: 41816.3
epoch:   14, step    0, Time: 11.4009s, gt_cnt:  40.0, et_cnt: 230.6 train_loss: 45.3
epoch:   14, step   10, Time: 30.5138s, gt_cnt: 120.0, et_cnt:  47.5 train_loss: 1633.0
epoch:   14, step   20, 

epoch:   18, step    0, Time: 11.3767s, gt_cnt:  40.0, et_cnt: 232.6 train_loss: 45.6
epoch:   18, step   10, Time: 30.2921s, gt_cnt: 120.0, et_cnt:  37.9 train_loss: 1632.5
epoch:   18, step   20, Time: 21.4629s, gt_cnt:  47.0, et_cnt:  94.9 train_loss: 2180.2
epoch:   18, step   40, Time: 35.9828s, gt_cnt:  38.0, et_cnt: 206.2 train_loss: 9439.9
epoch:   18, step   50, Time: 19.4215s, gt_cnt:  62.0, et_cnt:  50.2 train_loss: 9915.2
epoch:   18, step   60, Time: 22.1020s, gt_cnt: 560.0, et_cnt: 341.8 train_loss: 13230.0
epoch:   18, step   70, Time: 13.6087s, gt_cnt:  76.0, et_cnt:  38.6 train_loss: 14973.5
epoch:   18, step   90, Time: 32.6410s, gt_cnt:  85.0, et_cnt:  74.4 train_loss: 17461.4
epoch:   18, step  100, Time: 17.1723s, gt_cnt:  61.0, et_cnt:  46.0 train_loss: 18053.2
epoch:   18, step  110, Time: 15.0095s, gt_cnt:  31.0, et_cnt: 145.5 train_loss: 20712.0
epoch:   18, step  120, Time: 9.5916s, gt_cnt: 196.0, et_cnt:  98.7 train_loss: 22482.6
epoch:   18, step  130, Time:

...

epoch:   81, step    0, Time: 11.1261s, gt_cnt:  40.0, et_cnt: 107.0 train_loss: 44.1
epoch:   81, step   10, Time: 29.8078s, gt_cnt: 120.0, et_cnt:  37.3 train_loss: 1624.9
epoch:   81, step   20, Time: 21.2789s, gt_cnt:  47.0, et_cnt:  41.1 train_loss: 2167.1
epoch:   81, step   40, Time: 35.4005s, gt_cnt:  38.0, et_cnt: 150.7 train_loss: 9224.6
epoch:   81, step   50, Time: 19.4179s, gt_cnt:  62.0, et_cnt:  52.6 train_loss: 9704.5
epoch:   81, step   60, Time: 21.4592s, gt_cnt: 560.0, et_cnt:  89.1 train_loss: 12986.5
epoch:   81, step   70, Time: 13.2529s, gt_cnt:  76.0, et_cnt:  37.3 train_loss: 14728.1
epoch:   81, step   90, Time: 31.9025s, gt_cnt:  85.0, et_cnt:  46.8 train_loss: 17164.6
epoch:   81, step  100, Time: 16.4612s, gt_cnt:  61.0, et_cnt:  18.1 train_loss: 17750.3
epoch:   81, step  110, Time: 14.5243s, gt_cnt:  31.0, et_cnt:  41.0 train_loss: 20302.7
epoch:   81, step  120, Time: 9.3279s, gt_cnt: 196.0, et_cnt:  95.0 train_loss: 22027.8
epoch:   81, step  130, Time:

epoch:   85, step  110, Time: 14.6178s, gt_cnt:  31.0, et_cnt:  42.0 train_loss: 20198.8
epoch:   85, step  120, Time: 9.4046s, gt_cnt: 196.0, et_cnt:  91.3 train_loss: 21918.0
epoch:   85, step  130, Time: 19.6514s, gt_cnt:  42.0, et_cnt:  77.0 train_loss: 22644.3
epoch:   85, step  140, Time: 25.6200s, gt_cnt: 678.0, et_cnt: 274.1 train_loss: 29549.5
epoch:   85, step  150, Time: 19.6378s, gt_cnt: 1622.0, et_cnt: 1687.9 train_loss: 32621.2
epoch:   85, step  160, Time: 20.0299s, gt_cnt:  20.0, et_cnt:   7.9 train_loss: 34621.3
epoch:   85, step  170, Time: 15.3609s, gt_cnt: 227.0, et_cnt: 134.1 train_loss: 36141.0
epoch:   85, step  180, Time: 12.9455s, gt_cnt:  95.0, et_cnt: 122.4 train_loss: 38620.0
epoch:   85, step  190, Time: 9.9514s, gt_cnt: 273.0, et_cnt: 196.8 train_loss: 40457.6
epoch:   86, step    0, Time: 11.1642s, gt_cnt:  40.0, et_cnt:  92.2 train_loss: 43.8
epoch:   86, step   10, Time: 29.7009s, gt_cnt: 120.0, et_cnt:  37.7 train_loss: 1620.9
epoch:   86, step   20, T

epoch:   90, step    0, Time: 11.1408s, gt_cnt:  40.0, et_cnt:  88.1 train_loss: 43.7
epoch:   90, step   10, Time: 30.1219s, gt_cnt: 120.0, et_cnt:  40.1 train_loss: 1619.6
epoch:   90, step   20, Time: 21.2726s, gt_cnt:  47.0, et_cnt:  43.4 train_loss: 2162.4
epoch:   90, step   40, Time: 35.6319s, gt_cnt:  38.0, et_cnt: 128.5 train_loss: 9166.3
epoch:   90, step   50, Time: 19.3683s, gt_cnt:  62.0, et_cnt:  49.3 train_loss: 9647.3
epoch:   90, step   60, Time: 21.4790s, gt_cnt: 560.0, et_cnt:  96.7 train_loss: 12916.8
epoch:   90, step   70, Time: 13.2950s, gt_cnt:  76.0, et_cnt:  13.8 train_loss: 14637.6
epoch:   90, step   90, Time: 32.1000s, gt_cnt:  85.0, et_cnt:  47.0 train_loss: 17058.3
epoch:   90, step  100, Time: 16.4530s, gt_cnt:  61.0, et_cnt:  19.8 train_loss: 17643.3
epoch:   90, step  110, Time: 14.5334s, gt_cnt:  31.0, et_cnt:  33.4 train_loss: 20163.9
epoch:   90, step  120, Time: 9.3751s, gt_cnt: 196.0, et_cnt:  84.8 train_loss: 21879.8
epoch:   90, step  130, Time:

epoch:   94, step  110, Time: 14.6450s, gt_cnt:  31.0, et_cnt:  31.2 train_loss: 20132.1
epoch:   94, step  120, Time: 9.3394s, gt_cnt: 196.0, et_cnt:  88.6 train_loss: 21847.0
epoch:   94, step  130, Time: 19.8650s, gt_cnt:  42.0, et_cnt:  76.6 train_loss: 22573.2
epoch:   94, step  140, Time: 25.5998s, gt_cnt: 678.0, et_cnt: 280.8 train_loss: 29427.0
epoch:   94, step  150, Time: 19.7508s, gt_cnt: 1622.0, et_cnt: 1364.2 train_loss: 32478.0
epoch:   94, step  160, Time: 19.9887s, gt_cnt:  20.0, et_cnt:   8.7 train_loss: 34471.8
epoch:   94, step  170, Time: 15.3490s, gt_cnt: 227.0, et_cnt: 153.1 train_loss: 35989.4
epoch:   94, step  180, Time: 12.9856s, gt_cnt:  95.0, et_cnt: 128.1 train_loss: 38455.7
epoch:   94, step  190, Time: 9.9323s, gt_cnt: 273.0, et_cnt: 190.4 train_loss: 40283.5
epoch:   95, step    0, Time: 11.1592s, gt_cnt:  40.0, et_cnt:  98.7 train_loss: 44.0
epoch:   95, step   10, Time: 29.9390s, gt_cnt: 120.0, et_cnt:  42.2 train_loss: 1619.6
epoch:   95, step   20, T

epoch:   99, step   10, Time: 29.9828s, gt_cnt: 120.0, et_cnt:  44.3 train_loss: 1618.8
epoch:   99, step   20, Time: 21.3027s, gt_cnt:  47.0, et_cnt:  42.6 train_loss: 2161.6
epoch:   99, step   40, Time: 35.3969s, gt_cnt:  38.0, et_cnt: 121.7 train_loss: 9132.9
epoch:   99, step   50, Time: 19.0526s, gt_cnt:  62.0, et_cnt:  49.7 train_loss: 9613.4
epoch:   99, step   60, Time: 21.5371s, gt_cnt: 560.0, et_cnt: 105.0 train_loss: 12873.5
epoch:   99, step   70, Time: 13.2804s, gt_cnt:  76.0, et_cnt:  11.3 train_loss: 14592.1
epoch:   99, step   90, Time: 32.0898s, gt_cnt:  85.0, et_cnt:  47.9 train_loss: 17010.0
epoch:   99, step  100, Time: 16.4635s, gt_cnt:  61.0, et_cnt:  20.9 train_loss: 17593.9
epoch:   99, step  110, Time: 14.6046s, gt_cnt:  31.0, et_cnt:  25.1 train_loss: 20100.3
epoch:   99, step  120, Time: 9.3852s, gt_cnt: 196.0, et_cnt:  79.9 train_loss: 21810.4
epoch:   99, step  130, Time: 19.6487s, gt_cnt:  42.0, et_cnt:  71.5 train_loss: 22532.3
epoch:   99, step  140, Ti

In [15]:
log_text = f'BEST MAE: {best_mae}, BEST MSE: {best_mse}, BEST MAPE: {best_mape}'
log_print(log_text, color='green', attrs=['bold'])


BEST MAE: 999999, BEST MSE: 999999, BEST MAPE: 9999999


In [16]:
evaluate_model(best_mape_path, data_loader_val, is_cuda=is_cuda)

NameError: name 'best_mape_path' is not defined

In [ ]:
from models.smc.src.evaluate_model import analyse_loader

analyse_loader(data_loader_val)
analyse_loader(data_loader)

In [ ]:
best_mape_path = '../output/smc/saved_models/JHU/smc_0.0001.h5'

is_cuda = False
model2 = CrowdCounter(is_cuda=is_cuda)
network.weights_normal_init(model2, dev=0.01)

from models.csrnet_pytorch.src.network import load_net
load_net(best_mape_path, model)
model2.eval()

for i in [23,18,11,15,32,7]:
    test_image = data_loader_test.get_test_input(index=i)
    density_map_test = model2(test_image['data'])
    save_density_map(density_map_test.detach().numpy(), output_dir, fname=f'results_{i}.jpg')
    
    im_gray = cv2.imread(os.path.join(output_dir, f'results_{i}.jpg'), cv2.IMREAD_GRAYSCALE)
    im_jet = cv2.applyColorMap(im_gray, cv2.COLORMAP_JET)
    print(cv2.imwrite(os.path.join(output_dir, f'results_{i}.jpg'), im_jet))
    
    plt.imshow(cv2.imread(os.path.join(output_dir, f'results_{i}.jpg')))
    plt.show()